In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scikits.odes.ode import ode
import time
import random

#Import local functions
sys.path.append('../functions')
from functions import nonlinearity, repression, activation

In [2]:
def solve_point(x, func):
    n1, n2, theta1, theta2, k1, k2 = x
    
    alpha1 = 1E-5
    alpha2 = 1E-2
    kc=12.; km=10.; lam=1.93E-4; Vin=1.; e0=0.0467

    int_time = 5E4
    t = np.linspace(0, int_time, 100) 
    y0 = np.array([2290., 0., 0., 0., 0., 0.]) 

    def dual_control_sundials(t, y, ydot):
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
        ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = repression(y[1], k1, theta1, n1) + activation(y[1], k2, theta2, n2)

    def upstream_repression_sundials(t, y, ydot):
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = repression(y[1], k1, theta1, n1) - lam*y[2]
        ydot[3] = k2 - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = repression(y[1], k1, theta1, n1) + k2
    
    def downstream_activation_sundials(t, y, ydot):
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = k1 - lam*y[2]
        ydot[3] = activation(y[1], k2, theta2, n2) - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = activation(y[1], k2, theta2, n2) + k1

    def no_control(t, y, ydot):
        ydot[0] = Vin - lam*y[0] - e0*nonlinearity(y[0], kc, km) - y[2]*nonlinearity(y[0], kc, km)
        ydot[1] = y[2]*nonlinearity(y[0], kc, km) - y[3]*nonlinearity(y[1], kc, km) - lam*y[1]
        ydot[2] = k1 - lam*y[2]
        ydot[3] = k2 - lam*y[3]
        ydot[4] = (Vin -  y[3]*nonlinearity(y[1], kc, km))**2
        ydot[5] = k1 + k2


    def loss_biological(j1, j2, alpha1, alpha2):
        """Computes scalarized loss including genetic constraint and product production"""
        loss = alpha1*j1 + alpha2*j2
        return j1, j2, loss

    if func == 'dc':
        solution = ode('cvode', dual_control_sundials, old_api=False).solve(t, y0)
    elif func == 'ur':
        solution = ode('cvode', upstream_repression_sundials, old_api=False).solve(t, y0)
    elif func == 'da':
        solution = ode('cvode', downstream_activation_sundials, old_api=False).solve(t, y0)
    elif func == 'nc':
        solution = ode('cvode', no_control, old_api=False).solve(t, y0)
    j1, j2 = solution.values.y[-1, -2:]
    j1, j2, loss = loss_biological(j1, j2, alpha1=alpha1, alpha2=alpha2)
    return loss

In [4]:
def gen_ics(N):
    #Set random seed
    np.random.seed(2022)
    ics = []
    for i in range(N):
        k1 = random.uniform(1E-7,1E-3)
        k2 = random.uniform(1E-7,1E-3)
        n1 = random.uniform(1,4)
        n2 = random.uniform(1,4)
        theta1 = random.uniform(0.001,10)
        theta2 = random.uniform(0.001,10)

        init_conds = np.array([n1,n2,theta1, theta2, k1, k2])
        ics.append(init_conds)
    return ics

ics_100 = gen_ics(100)
ics_1k = gen_ics(1000)
ics_10k = gen_ics(10000)

In [26]:
func='nc'
ics = ics_10k
num_samples = 10000
num_epochs = 100
tolerance = 0.0004
patch_size = 100
step_sizes = np.array([0.1, 0.1, 0.01, 0.01, 0.0001, 0.0001])

start_time = time.time()
losses = []; k1_trace = []; k2_trace = []; n1_trace = []; n2_trace = []; theta1_trace = []; theta2_trace = []
for j in range(len(ics)):
    init_conds = ics[j]
    new_n1, new_n2, new_theta1, new_theta2, new_k1, new_k2 = init_conds
    loss = solve_point(init_conds, func)
    losses.append(loss)
    k1_trace.append(new_k1)
    k2_trace.append(new_k2)
    n1_trace.append(new_n1)
    n2_trace.append(new_n2)
    theta1_trace.append(new_theta1)
    theta2_trace.append(new_theta2)
elapsed_time = time.time() - start_time
results = pd.DataFrame({'circuit': func, 'num_samples': num_samples,'loss':losses,'k1_trace':k1_trace,'k2_trace':k2_trace,'n1_trace':n1_trace,'n2_trace':n2_trace,'theta1_trace':theta1_trace, 'theta2_trace':theta2_trace, 'time': elapsed_time})
results.to_csv(func+str(num_samples)+'.csv')